In [1]:
import pandas as pd
import csv
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import joblib
from config import user_name,password

In [41]:
# Creating 
btc = pd.read_csv("data/BTC-USD.csv")
btc = btc.drop(columns=["Adj Close","Open","High","Low","Volume"])
btc['Date']= pd.to_datetime(btc['Date'])
btc = btc.dropna()
btc_data = btc.iloc[::-1]
btc_data.head()

,Date,Close
1826,2021-10-08,54199.207031
1824,2021-10-06,55361.449219
1823,2021-10-05,51514.812500
1822,2021-10-04,49112.902344
1821,2021-10-03,48199.953125


In [3]:
btc_data = btc_data.rename(columns={'Close': 'Close_Btc'})
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data['Day_Btc'] = btc_data['Date'].dt.day_name()
btc_data = btc_data.sort_values(by="Date")
# btc_data

In [4]:
btc_data_clean = btc_data.loc[((btc_data['Day_Btc'] != "Saturday") & (btc_data['Day_Btc'] != 'Sunday'))]
# btc_data_clean

In [27]:
# btc_data_clean = btc_data_clean.drop(columns=["Day_Btc"])
btc_data_clean["BTC_Change%"] = btc_data_clean["Close_Btc"].pct_change()
btc_data_clean['BTC_Change%_lag'] = btc_data_clean["BTC_Change%"].shift(1)

# Construction direction of BITCOIN movement
btc_data_clean['BTC_dir'] = (btc_data_clean["BTC_Change%"]>0).astype(float)
btc_data_clean['BTC_dir_lag'] = btc_data_clean['BTC_dir'].shift(1)
btc_data_clean.head()

,Date,Close_Btc,BTC_Change%,BTC_Change%_lag,BTC_dir,BTC_dir_lag
2,2016-10-10,618.994019,NaN,NaN,0.0,NaN
3,2016-10-11,641.072021,0.035668,NaN,1.0,0.0
4,2016-10-12,636.192017,-0.007612,0.035668,0.0,1.0
5,2016-10-13,636.786011,0.000934,-0.007612,1.0,0.0
6,2016-10-14,640.377991,0.005641,0.000934,1.0,1.0


In [6]:
eth = pd.read_csv("data/ETH-USD.csv")
eth = eth.drop(columns=["Adj Close","Open","High","Low","Volume"])
eth['Date']= pd.to_datetime(eth['Date'])
eth_data = eth.dropna()
eth_data = eth_data.iloc[::-1]
# eth_data.head()

In [7]:
eth_data = eth_data.rename(columns={'Close': 'Close_Eth'})
eth_data['Date'] = pd.to_datetime(eth_data['Date'])
eth_data['Day_Eth'] = eth_data['Date'].dt.day_name()
eth_data = eth_data.sort_values(by="Date")
# eth_data

In [8]:
eth_data_clean = eth_data.loc[((eth_data['Day_Eth'] != "Saturday") & (eth_data['Day_Eth'] != 'Sunday'))]
# eth_data_clean

In [9]:
eth_data_clean = eth_data_clean.drop(columns=["Day_Eth"])
eth_data_clean["ETH_Change%"] = eth_data_clean["Close_Eth"].pct_change()
eth_data_clean['ETH_Change%_lag'] = eth_data_clean["ETH_Change%"].shift(1)

# Construction direction of Ethereum movement
eth_data_clean['ETH_dir'] = (eth_data_clean["ETH_Change%"]>0).astype(float)
eth_data_clean['ETH_dir_lag'] = eth_data_clean['ETH_dir'].shift(1)
# eth_data_clean.head()

In [10]:
gold = pd.read_csv("data/Gold-Futures.csv")
gold = gold.drop(columns=["Vol.","Change %","Open","High","Low"])
gold['Date']= pd.to_datetime(gold['Date'])
gold_data = gold.dropna()
# gold_data.head()

In [11]:
gold_data = gold_data.rename(columns={'Price': 'Price_Gold'})
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
gold_data['Day_Gold'] = gold_data['Date'].dt.day_name()
gold_data = gold_data.sort_values(by="Date")
gold_data = gold_data.reset_index(drop=True)
gold_data['Price_Gold'] = gold_data.Price_Gold.str.replace(',', '').astype(float)
# gold_data

In [12]:
gold_data_clean = gold_data.loc[((gold_data['Day_Gold'] != "Saturday") & (gold_data['Day_Gold'] != 'Sunday'))]
# gold_data_clean

In [13]:
gold_data_clean = gold_data_clean.drop(columns=["Day_Gold"])
gold_data_clean["Gold_Change%"] = gold_data_clean["Price_Gold"].pct_change()
gold_data_clean['Gold_Change%_lag'] = gold_data_clean["Gold_Change%"].shift(1)

# Construction direction of GOLD movement
gold_data_clean['Gold_Dir'] = (gold_data_clean["Gold_Change%"]>0).astype(float)
gold_data_clean['Gold_Dir_Lag'] = gold_data_clean['Gold_Dir'].shift(1)
gold_data_clean.head()

,Date,Price_Gold,Gold_Change%,Gold_Change%_lag,Gold_Dir,Gold_Dir_Lag
0,2016-09-08,1400.2,NaN,NaN,0.0,NaN
1,2016-09-09,1393.1,-0.005071,NaN,0.0,0.0
2,2016-09-12,1384.4,-0.006245,-0.005071,0.0,0.0
3,2016-09-13,1380.8,-0.002600,-0.006245,0.0,0.0
4,2016-09-14,1383.3,0.001811,-0.002600,1.0,0.0


In [14]:
snp = pd.read_csv("data/S&P-500.csv")
snp = snp.drop(columns=["Vol.","Change %","Open","High","Low"])
snp['Date']= pd.to_datetime(snp['Date'])
snp_data = snp.dropna()
# snp_data.head()

In [15]:
snp_data = snp_data.rename(columns={'Price': 'Price_Snp'})
snp_data['Date'] = pd.to_datetime(snp_data['Date'])
snp_data['Day_Snp'] = snp_data['Date'].dt.day_name()
snp_data = snp_data.sort_values(by="Date")
snp_data = snp_data.reset_index(drop=True)
snp_data['Price_Snp'] = snp_data.Price_Snp.str.replace(',', '').astype(float)
# snp_data

In [16]:
snp_data_clean = snp_data.loc[((snp_data['Day_Snp'] != "Saturday") & (snp_data['Day_Snp'] != 'Sunday'))]
# snp_data_clean

In [17]:
snp_data_clean = snp_data_clean.drop(columns=["Day_Snp"])
snp_data_clean["Snp_Change%"] = snp_data_clean["Price_Snp"].pct_change()
snp_data_clean['Snp_Change%_lag'] = snp_data_clean["Snp_Change%"].shift(1)

# Construction direction of S&P movement
snp_data_clean['Snp_dir'] = (snp_data_clean["Snp_Change%"]>0).astype(float)
snp_data_clean['Snp_dir_lag'] = snp_data_clean['Snp_dir'].shift(1)
# snp_data_clean

In [18]:
usbond = pd.read_csv("data/US-10-Year-Bond.csv")
usbond = usbond.drop(columns=["Change %","Open","High","Low"])
usbond['Date']= pd.to_datetime(usbond['Date'])
usbond_data = usbond.dropna()
# usbond_data.head()

In [19]:
usbond_data = usbond_data.rename(columns={'Price': 'Price_USB'})
usbond_data['Date'] = pd.to_datetime(usbond_data['Date'])
usbond_data['Day_USB'] = usbond_data['Date'].dt.day_name()
usbond_data = usbond_data.sort_values(by="Date")
usbond_data = usbond_data.reset_index(drop=True)
# usbond_data

In [20]:
usbond_data_clean = usbond_data.loc[((usbond_data['Day_USB'] != "Saturday") & (usbond_data['Day_USB'] != 'Sunday'))]
# usbond_data_clean

In [21]:
usbond_data_clean = usbond_data_clean.drop(columns=["Day_USB"])
usbond_data_clean["USB_Change%"] = usbond_data_clean["Price_USB"].pct_change()
usbond_data_clean['USB_Change%_lag'] = usbond_data_clean["USB_Change%"].shift(1)

# Construction direction of US BOND movement
usbond_data_clean['USB_dir'] = (usbond_data_clean["USB_Change%"]>0).astype(float)
usbond_data_clean['USB_dir_lag'] = usbond_data_clean['USB_dir'].shift(1)
# usbond_data_clean

In [30]:
bit_1 = btc_data_clean.merge(eth_data_clean, how='left', on='Date')
bit_2 = bit_1.merge(snp_data_clean, how='left', on='Date')
bit_3 = bit_2.merge(usbond_data_clean, how='left', on='Date')
bit_3 = bit_3.dropna()
bit_3 = bit_3.reset_index(drop=True)

bit_3 = bit_3[['Date', 'BTC_dir', 'BTC_dir_lag', 'BTC_Change%_lag', 'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
bit_3.head()

,Date,BTC_dir,BTC_dir_lag,BTC_Change%_lag,USB_Change%_lag,Snp_Change%_lag,ETH_Change%_lag
0,2016-10-12,0.0,1.0,0.035668,0.001133,-0.012447,0.002569
1,2016-10-13,1.0,0.0,-0.007612,0.002264,0.001147,0.012541
2,2016-10-14,1.0,1.0,0.000934,-0.014116,-0.003099,0.002472
3,2016-10-17,0.0,1.0,0.005641,0.033792,0.000202,-0.002073
4,2016-10-18,0.0,0.0,-0.001850,-0.020499,-0.003038,0.005595


In [25]:
gold_1 = gold_data_clean.merge(usbond_data_clean, how='left', on='Date')
gold_2 = gold_1.merge(snp_data_clean, how='left', on='Date')
gold_3 = gold_2.merge(eth_data_clean, how='left', on='Date')
gold_3 = gold_3.dropna()
gold_3 = gold_3.reset_index(drop=True)

gold_3 = gold_3[['Date', 'Gold_Dir', 'Gold_Dir_Lag', 'Gold_Change%_lag', 'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
gold_3.tail()

,Date,Gold_Dir,Gold_Dir_Lag,Gold_Change%_lag,USB_Change%_lag,Snp_Change%_lag,ETH_Change%_lag
1245,2021-09-30,1.0,0.0,-0.008403,-0.014230,0.001569,0.016331
1246,2021-10-01,1.0,1.0,0.019792,-0.020997,-0.011910,0.052060
1247,2021-10-04,1.0,1.0,0.000797,-0.018097,0.011494,0.101889
1248,2021-10-05,0.0,1.0,0.005232,0.010922,-0.012988,0.021942
1249,2021-10-06,1.0,0.0,-0.003790,0.031735,0.010524,0.040954


In [34]:
train_bit = bit_3.iloc[:-100, :]
test_bit = bit_3.iloc[-100:, :]
# train_bit.tail(), test_bit.head()

x_train_bit = train_bit[['BTC_dir_lag', 'BTC_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_train_bit = train_bit['BTC_dir']
x_test_bit = test_bit[['BTC_dir_lag', 'BTC_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_test_bit = test_bit['BTC_dir']


In [166]:
train_gold = gold_3.iloc[:-100, :]
test_gold = gold_3.iloc[-100:, :]
train_gold.tail(), test_gold.head()

x_train_gold = train_gold[['Gold_Dir_Lag', 'Gold_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_train_gold = train_gold['Gold_Dir']
x_test_gold = test_gold[['Gold_Dir_Lag', 'Gold_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_test_gold = test_gold['Gold_Dir']


In [192]:
classifier = LogisticRegression(max_iter=1000)
classifier = classifier.fit(x_train_gold, y_train_gold)

In [193]:
features_gold = x_train_gold.columns.tolist()
selector = RFECV(estimator=classifier, cv=5, step=1)
selector.fit(x_train_gold, y_train_gold)

RFECV(cv=5, estimator=LogisticRegression(max_iter=1000))

In [195]:
selected_features_gold = sorted(zip(selector.ranking_, features_gold))
features_rank_gold = pd.DataFrame(selected_features_gold, columns=['Ranking', 'Feature'])
features_rank_gold = features_rank_gold.set_index('Feature')
features_rank_gold

,Ranking
Feature,
ETH_Change%_lag,1
USB_Change%_lag,1
Gold_Dir_Lag,2
Gold_Change%_lag,3
Snp_Change%_lag,4


In [196]:
ranked_features_gold = []
for s in selected_features_gold:
    if s[0] < 3:
        ranked_features_gold.append(s[1])

In [198]:
x_train_gold_sel = x_train_gold[ranked_features_gold]
x_test_gold_sel = x_test_gold[ranked_features_gold]

In [257]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
param_grid = {'max_features':['auto', 'sqrt', 'log2'],
           'class_weight':['balanced', 'balanced_subsample'],
             'criterion':['gini', 'entropy'],
             'min_samples_leaf': [1,5,15,25],
             'n_estimators': [10,100,500,1000,2000,3000]}

grid = GridSearchCV(rf, param_grid, n_jobs=-1)

In [258]:
grid.fit(x_train_gold, y_train_gold)

GridSearchCV(estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 5, 15, 25],
                         'n_estimators': [10, 100, 500, 1000, 2000, 3000]})

In [259]:
max_features = grid.best_params_['max_features']
class_weight = grid.best_params_['class_weight']
criterion = grid.best_params_['criterion']
min_samples_leaf = grid.best_params_['min_samples_leaf']
# min_samples_split = grid.best_params_['min_samples_split']
n_estimators = grid.best_params_['n_estimators']

In [260]:
tuned_model_gold = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, class_weight=class_weight, criterion=criterion, 
                                          min_samples_leaf=min_samples_leaf, random_state=42)
tuned_model_gold.fit(x_train_gold, y_train_gold)
rf_training_score = round(tuned_model_gold.score(x_train_gold, y_train_gold)*100,3)
rf_test_score = round(tuned_model_gold.score(x_test_gold, y_test_gold)*100,3)

print(f"Training Data Score: {rf_test_score} %")
print(f"Testting Data Score: {rf_test_score} %")

Training Data Score: 50.0 %
Testting Data Score: 50.0 %
